In [1]:
import torch
torch.cuda.empty_cache()

In [3]:
#file variables

#pretrained_bert = 'cimm-kzn/endr-bert'
pretrained_bert = 'allenai/scibert_scivocab_uncased'
#pretrained_bert = 'dmis-lab/biobert-base-cased-v1.2'
#pretrained_bert = 'bert-large-uncased'
#pretrained_bert = 'microsoft/BiomedNLP-BiomedBERT-large-uncased-abstract'
#pretrained_bert = 'microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext'
#pretrained_bert = 'dmis-lab/biobert-v1.1'
#pretrained_bert = 'bert-base-uncased'
#pretrained_bert = 'SpanBERT/spanbert-base-cased
#pretrained_bert = 'google/pegasus-xsum'
#pretrained_bert = 'facebook/bart-base'
#pretrained_bert = 'emilyalsentzer/Bio_ClinicalBERT'
#pretrained_bert = 'allenai/scibert_scivocab cased'


#TRAINING FILE
use_huggingface = True
training_file = "**training**.csv"
# training_file = "**training**.csv"
column1_AEtext = "text" #"Example"
column2_AEflag = "label" #"AE Flag"
train_test_split_percent = 0.8
balance_multiplier = 1.0
#batch_size = 3
balance_me = True
cross_val_number = 5
weight_decay = 0.01
#balance_me = False

#TEST FILE
test_file = "**test**.csv"
# test_file = "**test**.csv"
test_column1_AEtext = "Example"
test_column2_AEflag = "AE Flag"

In [4]:
import os 
os.environ['WANDB_NOTEBOOK_NAME'] = 'BERTcomparison.ipynb'

In [5]:
import wandb
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find BERTcomparison.ipynb.
wandb: Currently logged in as: jon-l-collins. Use `wandb login --relogin` to force relogin


True

In [6]:
sweep_config = {
    'method': 'bayes'
    }

In [7]:
metric = {
        'name': 'val_loss',
        'goal': 'minimize'
}
sweep_config['metric'] = metric

In [8]:
import datetime
print(str(datetime.datetime.now()))

2024-02-02 21:17:49.860864


In [9]:
parameters_dict = {
    'learning_rate': {
        'values': [5e-6]
    },
    'batch_size': {
        'values': [12]
        },
    
    'dropout': {
          'values': [0.1,0.4,0.5]
        },
    
    'epochs': {
        'values': [14]
    }   
 
}
sweep_config['parameters'] = parameters_dict
pretrained_bert2 = pretrained_bert.replace('/','')
sweep_id = wandb.sweep(sweep_config, project=f"{pretrained_bert2}_{str(datetime.datetime.now())[:10]}")

Create sweep with ID: vpaxo9s7
Sweep URL: https://wandb.ai/jon-l-collins/allenaiscibert_scivocab_uncased_2024-02-02/sweeps/vpaxo9s7


In [10]:
import torch
#from tqdm.notebook import tqdm
#from tqdm import tqdm
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, AutoTokenizer, AutoModelForSequenceClassification,AutoConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup
import numpy as np
import time
import datetime
import random
import os
import pandas as pd
from tqdm.notebook import tqdm

In [11]:
from datasets import load_dataset
use_huggingface = True
if use_huggingface == True:
    dataset = load_dataset("ade_corpus_v2","Ade_corpus_v2_classification")
else:
    dataset = pd.read_csv(training_file, encoding='unicode_escape')

In [12]:
if use_huggingface == True:
    #convert to pandas dataframe
    df = pd.DataFrame(dataset['train'])

In [13]:
df[column2_AEflag].value_counts()

label
0    16695
1     6821
Name: count, dtype: int64

In [14]:
num_AEs = df[column2_AEflag].value_counts()[1]
balance_number = int(num_AEs * balance_multiplier)

# num_AEs = 500
# balance_number = int(num_AEs * balance_multiplier)

In [15]:
print(f"num_AEs = {num_AEs}\nbalance_number = {balance_number}\nbalance_multiplier = {balance_multiplier}")

num_AEs = 6821
balance_number = 6821
balance_multiplier = 1.0


In [16]:
if(balance_me == True):
    d_NoAE = df[df[column2_AEflag]==0]
    shuffle_d_NoAE = d_NoAE.sample(frac=1,random_state=42)[:balance_number]
    d_AE = df[df[column2_AEflag]==1]
    #d_AE = df[df[column2_AEflag]==1][:500]
    df = pd.concat([shuffle_d_NoAE,d_AE])

In [17]:
df[column2_AEflag].value_counts()

label
0    6821
1    6821
Name: count, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split
train_data,test = train_test_split(df, test_size=1-train_test_split_percent,stratify=df[column2_AEflag], random_state=42)

c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [20]:
train_data.shape, test.shape

((10913, 2), (2729, 2))

In [21]:
from torch.utils.data import TensorDataset, random_split
import random
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
def process_data(df_train,df_test):
    #tokenizer = BertTokenizer.from_pretrained(pretrained_bert, do_lower_case=True)
    tokenizer = AutoTokenizer.from_pretrained(pretrained_bert, add_prefix_space=True, use_fast=False)
    encoded_data_train = tokenizer.batch_encode_plus(df_train[column1_AEtext].values,
                                                    add_special_tokens=True,
                                                    return_attention_mask=True,
                                                    pad_to_max_length=True,
                                                    max_length=512,
                                                    return_tensors='pt')
    encoded_data_val = tokenizer.batch_encode_plus(df_test[column1_AEtext].values,
                                                    add_special_tokens=True,
                                                    return_attention_mask=True,
                                                    pad_to_max_length=True,
                                                    max_length=512,
                                                    return_tensors='pt')
    input_ids_train = encoded_data_train['input_ids']
    attention_masks_train = encoded_data_train['attention_mask']
    #labels = torch.tensor(dataset['AE Flag'].replace(label_dict).values)
    labels_train = torch.tensor(df_train[column2_AEflag].values)    
    input_ids_val = encoded_data_val['input_ids']
    attention_masks_val = encoded_data_val['attention_mask']
    #labels = torch.tensor(dataset['AE Flag'].replace(label_dict).values)
    labels_val = torch.tensor(df_test[column2_AEflag].values)
    dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
    dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
    batch_size = wandb.config.batch_size
    dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batch_size)
    dataloader_validation = DataLoader(dataset_val, sampler = SequentialSampler(dataset_val), batch_size=batch_size)
    return dataloader_train,dataloader_validation

In [22]:
def ret_model():
    
    configuration = AutoConfig.from_pretrained(pretrained_bert)
    configuration.hidden_dropout_prob = wandb.config.dropout
    configuration.attention_probs_dropout_prob = wandb.config.dropout
    configuration.num_labels = 2
    configuration.output_attentions = False
    configuration.output_hidden_states = False

    model = AutoModelForSequenceClassification.from_pretrained(pretrained_bert,config=configuration)
    return model

In [23]:
from transformers import AdamW, get_linear_schedule_with_warmup

def ret_optim(model):
    print(f"learning_rate = {wandb.config.learning_rate}")
    optimizer = AdamW(model.parameters(),
                      lr = wandb.config.learning_rate, 
                      eps = 1e-8,weight_decay=weight_decay)
    return optimizer

In [24]:
def ret_scheduler(optimizer, dataloader_train):
    epochs = wandb.config.epochs
    total_steps = len(dataloader_train) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, 
                                                num_training_steps = total_steps)
    return scheduler

In [25]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [26]:
import random
import numpy as np

In [27]:
from sklearn.metrics import f1_score

def f1_score_func(preds,labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [28]:
def classify(text,tokenizer,device,model):
    encoded_input = tokenizer(text, return_tensors="pt",truncation=True,max_length=512,padding=True)
    encoded_input.to(device)
    with torch.no_grad():
        output = model(**encoded_input)
    res = output.logits[0].detach().to('cpu').numpy()
    return np.argmax(res)

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [29]:
def classificationreporter(model,model_name):
    #tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True, use_fast=False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    results=[]
    for text in test[column1_AEtext].values:
        results.append(classify(text,tokenizer,device,model))
    ground_truth=test[column2_AEflag].values
    conf = confusion_matrix(ground_truth, results)
    ac=accuracy_score(ground_truth, results)
    #print(conf)
    print('Accuracy Score on test data:',ac)
    return conf,ground_truth,results,ac

In [30]:
def classificationreporterval(model,model_name,val_df):
    #tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)
    tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True, use_fast=False)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    results=[]
    for text in val_df[column1_AEtext].values:
        results.append(classify(text,tokenizer,device,model))
    ground_truth=val_df[column2_AEflag].values
    conf = confusion_matrix(ground_truth, results)
    ac=accuracy_score(ground_truth, results)
    #print(conf)
    print('Accuracy Score on validation data:',ac)
    return conf,ground_truth,results,ac

In [31]:
from sklearn.metrics import recall_score,f1_score

In [33]:
import wandb
def train(model,train_df,val_df):
    epochs = wandb.config.epochs
    dataloader_train, dataloader_validation = process_data(train_df,val_df)
    total_val_loss=[]
    total_val_f1=[]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    for epoch_i in range(0, epochs):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')
        optimizer = ret_optim(model)
        scheduler = ret_scheduler(optimizer, dataloader_train)
        training_stats = []
        t0 = time.time()
        total_train_loss = 0
        model.train()   
        for step, batch in enumerate(dataloader_train):
            if step % 200 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(dataloader_train), elapsed))
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            model.zero_grad()
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask,
                            labels=b_labels)
            loss = outputs[0]
            total_train_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        avg_train_loss = total_train_loss / len(dataloader_train)
        training_time = format_time(time.time() - t0)
        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))
        print("")
        model.eval()
        total_eval_loss = 0
        predictions, true_vals = [], []
        #nb_eval_steps = 0
        for batch in dataloader_validation:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():
                outputs = model(b_input_ids,
                                token_type_ids=None,
                                attention_mask=b_input_mask,
                                labels=b_labels)
                loss=outputs[0]
                logits=outputs[1]
            total_eval_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()
            predictions.append(logits)
            true_vals.append(label_ids)

        predictions = np.concatenate(predictions, axis=0) #added
        true_vals = np.concatenate(true_vals, axis=0) #added

        val_f1 = f1_score_func(predictions, true_vals)
        total_val_f1.append(val_f1) #added
        avg_val_loss = total_eval_loss / len(dataloader_validation)
        total_val_loss.append(avg_val_loss)
    conf_val,ground_truth_val,results_val,accuracy_score_val=classificationreporterval(model,pretrained_bert, val_df)
    recall_val = recall_score(ground_truth_val, results_val, pos_label=1)
    f1_val = f1_score(ground_truth_val, results_val, pos_label=1)
    print('Recall value for validation data is :',recall_val)
    print('F1 value for validation data is :',f1_val)
    print('Confusion Matrix :')
    print(conf_val)
    print('Report on val data: ')
    print(classification_report(ground_truth_val, results_val))
    print("Running cumulative Classification Report on test data...") #print at end of each fold
    conf,ground_truth,results,accuracy_score=classificationreporter(model,model_name=pretrained_bert)  
    recall = recall_score(ground_truth, results, pos_label=1)
    f1 = f1_score(ground_truth, results, pos_label=1)
    #recall_test.append(recall)
    #f1_test.append(f1)
    print('Recall value for test data is :',recall)
    print('F1 value for test data is :',f1)
    print('Confusion Matrix :')  
    print(conf)
    print('Report on test data: ')
    print(classification_report(ground_truth, results))                           
    return total_val_loss[-1],recall_val,f1_val,accuracy_score_val,conf_val,recall,f1,conf,accuracy_score
    #val_loss,recall_val,f1_val,accuracy_score_val,conf_val,recall,f1_test,conf,accuracy_score=train(model,train_df,val_df)

In [34]:
import numpy as np
from sklearn.model_selection import KFold
def train_folds():
    kf = KFold(n_splits=cross_val_number, random_state=42, shuffle=True)
    wandb.init()
    val=[]
    f1=[]
    rec_val=[]
    f1_v=[]
    acc_score_val=[]
    rec=[]
    confusion_val=[]
    confusion_test=[]
    acc_score=[]
    for fold,(train_index, val_index) in enumerate(kf.split(train_data)):
        print('======== Fold {:} / {:} ========'.format(fold+1,cross_val_number))
        train_df = train_data.iloc[train_index]
        val_df = train_data.iloc[val_index]
        model=ret_model()
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        val_loss,recall_val,f1_val,accuracy_score_val,conf_val,recall,f1_test,conf,accuracy_score=train(model,train_df,val_df)
        val.append(val_loss)
        f1.append(f1_test)
        rec_val.append(recall_val)
        f1_v.append(f1_val)
        acc_score_val.append(accuracy_score_val)
        rec.append(recall)
        confusion_val.append(np.array(conf_val))
        confusion_test.append(np.array(conf))
        acc_score.append(accuracy_score)
    
    f1_avg=sum(f1_v)/len(f1_v)
    val_avg=sum(val)/len(val)
    avg_acc_val=sum(acc_score_val)/len(acc_score_val)
    avg_rec_val=sum(rec_val)/len(rec_val)
    avg_acc=sum(acc_score)/len(acc_score)
    avg_recall=sum(rec)/len(rec)
    f1_test=sum(f1)/len(f1)
    average_conf_val=np.mean(confusion_val,axis=0)
    average_conf_test=np.mean(confusion_test,axis=0)
    print('Average Confusion Matrix for validation data:')
    print(average_conf_val)   
    print('Average Confusion Matrix for test data:')
    print(average_conf_test)    
    wandb.log({"val_loss":val_avg})
    wandb.log({"recall_validation": avg_rec_val})
    wandb.log({"f1_validation": f1_avg})
    wandb.log({"recall_test": avg_recall})
    wandb.log({"f1_test": f1_test})
    wandb.log({"Accuracy on test data": avg_acc})
    wandb.log({"Accuracy on validation data": avg_acc_val})

In [35]:
wandb.agent(sweep_id, train_folds)

wandb: Agent Starting Run: 70fri1y6 with config:
wandb: 	batch_size: 12
wandb: 	dropout: 0.4
wandb: 	epochs: 14
wandb: 	learning_rate: 5e-06
wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find BERTcomparison.ipynb.


======== Fold 1 / 5 ========


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_le


======== Epoch 1 / 14 ========
Training...
learning_rate = 5e-06


c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

  Batch   200  of    728.    Elapsed: 0:00:58.
  Batch   400  of    728.    Elapsed: 0:01:57.
  Batch   600  of    728.    Elapsed: 0:02:55.

  Average training loss: 0.55
  Training epoch took: 0:03:33


======== Epoch 2 / 14 ========
Training...
learning_rate = 5e-06


c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  Batch   200  of    728.    Elapsed: 0:00:59.
  Batch   400  of    728.    Elapsed: 0:02:03.
  Batch   600  of    728.    Elapsed: 0:03:05.

  Average training loss: 0.36
  Training epoch took: 0:03:43


======== Epoch 3 / 14 ========
Training...
learning_rate = 5e-06


c:\Users\jonlc\anaconda3\envs\bestpytorch\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  Batch   200  of    728.    Elapsed: 0:00:59.
  Batch   400  of    728.    Elapsed: 0:01:59.
  Batch   600  of    728.    Elapsed: 0:02:59.

  Average training loss: 0.30
  Training epoch took: 0:03:37

